In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Define the image dimensions and batch size

img_height = 224
img_width = 224
batch_size = 16

# Define the train, validation, and test data directories
train_dir = 'OCT2017_/train'
val_dir = 'OCT2017_/val'
test_dir = 'OCT2017_/test'

# Use the ResNet152V2 model as the base model
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


234545216/234545216 [==============================] - 29s 0us/step


In [3]:
# Add a custom output layer for the four classes
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
# Define the data augmentation for the training set
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# Define the validation and test data generators
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:

# Create the generators for the training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [6]:
# Train the model on the training set, using the validation set for validation
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size
)

Epoch 1/10
5217/5217 [==============================] - 13234s 3s/step - loss: 0.7841 - accuracy: 0.7621 - val_loss: 0.3283 - val_accuracy: 0.9062
Epoch 2/10
5217/5217 [==============================] - 13221s 3s/step - loss: 0.5207 - accuracy: 0.8104 - val_loss: 0.2591 - val_accuracy: 0.8750
Epoch 3/10
5217/5217 [==============================] - 13256s 3s/step - loss: 0.4876 - accuracy: 0.8235 - val_loss: 0.3822 - val_accuracy: 0.8750
Epoch 4/10
5217/5217 [==============================] - 13267s 3s/step - loss: 0.4733 - accuracy: 0.8300 - val_loss: 0.2252 - val_accuracy: 0.9062
Epoch 5/10
5217/5217 [==============================] - 13264s 3s/step - loss: 0.4581 - accuracy: 0.8345 - val_loss: 0.2017 - val_accuracy: 0.9062
Epoch 6/10
5217/5217 [==============================] - 13250s 3s/step - loss: 0.4538 - accuracy: 0.8384 - val_loss: 0.2225 - val_accuracy: 0.9375
Epoch 7/10
5217/5217 [==============================] - 15216s 3s/step - loss: 0.4423 - accuracy: 0.8409 - val_loss: 0

In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


61/61 [==============================] - 192s 3s/step - loss: 0.2286 - accuracy: 0.9143
Test accuracy: 0.91425621509552
